# 1. Define the Problem

From https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

I like to use a three step process to define the problem. I like to move quickly and I use this mini process to see the problem from a few different perspectives very quickly:

* Step 1: What is the problem? Describe the problem informally and formally and list assumptions and similar problems.
* Step 2: Why does the problem need to be solved? List your motivation for solving the problem, the benefits a solution provides and how the solution will be used.
* Step 3: How would I solve the problem? Describe how the problem would be solved manually to flush domain knowledge.


## 1.1 What is the problem? 
Given a set of metrics organised by type and country, we need to predict a subset of these metrics for one year ahead and five years ahead.
The data is quite sparse and very few entries have values for the complete time range of 1972-2007. It also appears that metric names may not be consistent between countries. The dataset will require a lot of cleaning before we can use it.

## 1.2 Why does the problem need to be solved?

### 1.2.1 From a global perspective
* The United Nations millenium development goals are a way of measuring the level of development of countries around the world in terms of important metrics such as poverty levels and female empowerment. It would be very useful to be able to predict when a given country might reach the levels the UN has set as targets.

### 1.2.2 From a study group perspective
* This dataset gives us a chance to work with time-series data (which we have not addressed as a group before)
* The dataset is quite sparse with many missing values and possible inconsistent fields. A successfull result with this will give us badly needed experience with data cleaning.

## 1.3 How would I solve the problem?

see README